In [14]:
!pip install pulp

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pulp

In [16]:
df = pd.read_csv("Indian_College_Student_Dorm_Items_Expanded (1).csv")

In [17]:
df.columns

Index(['Item', 'Monetary Value (INR)', 'Weight (kg)', 'Volume (liters)',
       'Age (years)', 'Depreciation per Year (%)', 'Airline Baggage Type'],
      dtype='object')

In [18]:
df.columns = df.columns.str.lower().str.replace(r"\s*\(.*?\)", "", regex=True).str.strip().str.replace(" ", "_")

df.head()

,item,monetary_value,weight,volume,age,depreciation_per_year,airline_baggage_type
0,Laptop,50000,2.0,5.0,2,15,Hand Baggage
1,Smartphone,20000,0.5,0.5,1,20,Hand Baggage
2,Backpack,3000,1.5,8.0,1,10,Hand Baggage
3,Desk Lamp,1500,1.0,4.0,2,10,Check-in Baggage
4,Bookshelf,5000,10.0,60.0,3,5,Check-in Baggage


In [19]:
df['current_value'] = (df['monetary_value']*(1-df['depreciation_per_year']/100)**df['age']).astype(int)

In [20]:
df['cabin_safe'] = df['airline_baggage_type']=='Hand Baggage'
df['checkin_safe'] = df['airline_baggage_type']=='Check-in Baggage'
df['movers_safe'] = (df['item']!='Laptop') & (df['item']!='Smartphone')

df.head()

,item,monetary_value,weight,volume,age,depreciation_per_year,airline_baggage_type,current_value,cabin_safe,checkin_safe,movers_safe
0,Laptop,50000,2.0,5.0,2,15,Hand Baggage,36124,True,False,False
1,Smartphone,20000,0.5,0.5,1,20,Hand Baggage,16000,True,False,False
2,Backpack,3000,1.5,8.0,1,10,Hand Baggage,2700,True,False,True
3,Desk Lamp,1500,1.0,4.0,2,10,Check-in Baggage,1215,False,True,True
4,Bookshelf,5000,10.0,60.0,3,5,Check-in Baggage,4286,False,True,True


In [21]:
# Assumptions
cabin_cap = {'max_weight': 7, 'max_volume': 45}
checkin_cap = {'max_weight': 23, 'max_volume': 50}
MOVERS_RATE = 2
SCALE = 2

In [22]:
import pandas as pd

def dp_packing_with_assignments(df, cabin_cap, checkin_cap, cost_per_volume, scale=2):
    """
    Dynamic Programming with assignment tracking.

    Returns:
    - best_state: (used_w1, used_v1, used_w2, used_v2)
    - best_value: maximum net value
    - best_assign: list of assignments per item: 0=cabin, 1=check-in, 2=movers
    """
    # Scale capacities
    W1 = int(cabin_cap['max_weight'] * scale)
    V1 = int(cabin_cap['max_volume'] * scale)
    W2 = int(checkin_cap['max_weight'] * scale)
    V2 = int(checkin_cap['max_volume'] * scale)

    # DP states: key=(cw,cv,tw,tv), value=(net_value, assign_list)
    states = {(0, 0, 0, 0): (0.0, [])}

    for idx, row in df.iterrows():
        w = int(row['weight'] * scale)
        v = int(row['volume'] * scale)
        val = row['current_value']
        mover_val = val - cost_per_volume * row['volume']

        new_states = dict(states)  # shallow copy

        for (cw, cv, tw, tv), (acc_val, assign_list) in states.items():
            # Option 0: Cabin
            if row['cabin_safe'] and cw + w <= W1 and cv + v <= V1:
                key = (cw + w, cv + v, tw, tv)
                new_val = acc_val + val
                new_assign = assign_list + [0]
                if key not in new_states or new_val > new_states[key][0]:
                    new_states[key] = (new_val, new_assign)

            # Option 1: Check-in
            if row['checkin_safe'] and tw + w <= W2 and tv + v <= V2:
                key = (cw, cv, tw + w, tv + v)
                new_val = acc_val + val
                new_assign = assign_list + [1]
                if key not in new_states or new_val > new_states[key][0]:
                    new_states[key] = (new_val, new_assign)

            # Option 2: Movers
            if row['movers_safe']:
                key = (cw, cv, tw, tv)
                new_val = acc_val + mover_val
                new_assign = assign_list + [2]
                if key not in new_states or new_val > new_states[key][0]:
                    new_states[key] = (new_val, new_assign)

        states = new_states

    # Extract best
    best_state, (best_value, best_assign) = max(states.items(), key=lambda kv: kv[1][0])
    return best_state, best_value, best_assign


best_state, best_value, best_assign = dp_packing_with_assignments(
    df, cabin_cap, checkin_cap, MOVERS_RATE, scale=SCALE
)

# Map numeric codes to text
choice_map = {0: 'Cabin', 1: 'Check-in', 2: 'Movers'}
df['assignment'] = [choice_map[c] for c in best_assign]

# Print results
print("Per-item assignment:")
print(df[['item','assignment']].to_string(index=False))
print(f"\nBest used capacities (cabin_w, cabin_v, checkin_w, checkin_v): {best_state}")
print(f"Maximum net value: ₹{best_value:.2f}")

Per-item assignment:
               item assignment
             Laptop      Cabin
         Smartphone      Cabin
           Backpack      Cabin
          Desk Lamp     Movers
          Bookshelf     Movers
        Study Table     Movers
              Chair     Movers
           Mattress   Check-in
       Bedsheet Set     Movers
             Pillow     Movers
     Laundry Basket     Movers
           Wardrobe     Movers
             Mirror     Movers
        Alarm Clock      Cabin
         Headphones      Cabin
     Extension Cord     Movers
       Water Bottle     Movers
         Tiffin Box      Cabin
   Fan (Table/Wall)     Movers
     Stationery Set      Cabin
Portable Hard Drive      Cabin
            Printer     Movers
    Electric Kettle     Movers
            Shirt 1     Movers
            Shirt 2     Movers
            Shirt 3     Movers
            Shirt 4      Cabin
            Shirt 5      Cabin
             Pant 1      Cabin
             Pant 2      Cabin
             Pant 

In [23]:
# Map numeric codes to text
choice_map = {0: 'Cabin', 1: 'Check-in', 2: 'Movers'}
df['assignment'] = [choice_map[c] for c in best_assign]

# Print per‐item assignment
print("Per-item assignment:")
print(df[['item','assignment']].to_string(index=False))

# Unpack and scale down best_state for final display
cw, cv, tw, tv = best_state
scale_factor = SCALE  # or just 2 if you're not changing it
used_cabin_w = cw / scale_factor
used_cabin_v = cv / scale_factor
used_check_w = tw / scale_factor
used_check_v = tv / scale_factor

# Print in the requested format
print(f"\nUsed Cabin Weight: {used_cabin_w:.1f} kg / {cabin_cap['max_weight']} kg")
print(f"Used Cabin Volume: {used_cabin_v:.1f} L / {cabin_cap['max_volume']} L")
print(f"Used Check-in Weight: {used_check_w:.1f} kg / {checkin_cap['max_weight']} kg")
print(f"Used Check-in Volume: {used_check_v:.1f} L / {checkin_cap['max_volume']} L")

print(f"\nMaximum Net Value (₹): {best_value:.2f}")

Per-item assignment:
               item assignment
             Laptop      Cabin
         Smartphone      Cabin
           Backpack      Cabin
          Desk Lamp     Movers
          Bookshelf     Movers
        Study Table     Movers
              Chair     Movers
           Mattress   Check-in
       Bedsheet Set     Movers
             Pillow     Movers
     Laundry Basket     Movers
           Wardrobe     Movers
             Mirror     Movers
        Alarm Clock      Cabin
         Headphones      Cabin
     Extension Cord     Movers
       Water Bottle     Movers
         Tiffin Box      Cabin
   Fan (Table/Wall)     Movers
     Stationery Set      Cabin
Portable Hard Drive      Cabin
            Printer     Movers
    Electric Kettle     Movers
            Shirt 1     Movers
            Shirt 2     Movers
            Shirt 3     Movers
            Shirt 4      Cabin
            Shirt 5      Cabin
             Pant 1      Cabin
             Pant 2      Cabin
             Pant 

In [24]:
# Define problem
model = pulp.LpProblem("PackingOptimization", pulp.LpMaximize)

# Decision variables
x_cabin = pulp.LpVariable.dicts('cabin', df.index, cat='Binary')
x_checkin = pulp.LpVariable.dicts('checkin', df.index, cat='Binary')
x_mover = pulp.LpVariable.dicts('mover', df.index, cat='Binary')

# Objective
model += pulp.lpSum(
    df.loc[i, 'current_value'] * (x_cabin[i] + x_checkin[i]) +
    (df.loc[i, 'current_value'] - MOVERS_RATE * df.loc[i, 'volume']) * x_mover[i]
    for i in df.index
)

# Constraints
# Each item must go exactly one way
for i in df.index:
    model += x_cabin[i] + x_checkin[i] + x_mover[i] == 1

# Capacity constraints
model += pulp.lpSum(df.loc[i, 'weight'] * x_cabin[i] for i in df.index) <= cabin_cap['max_weight']
model += pulp.lpSum(df.loc[i, 'volume'] * x_cabin[i] for i in df.index) <= cabin_cap['max_volume']
model += pulp.lpSum(df.loc[i, 'weight'] * x_checkin[i] for i in df.index) <= checkin_cap['max_weight']
model += pulp.lpSum(df.loc[i, 'volume'] * x_checkin[i] for i in df.index) <= checkin_cap['max_volume']

# Safety constraints
for i in df.index:
    if not df.loc[i, 'cabin_safe']:
        model += x_cabin[i] == 0
    if not df.loc[i, 'checkin_safe']:
        model += x_checkin[i] == 0
    if not df.loc[i, 'movers_safe']:
        model += x_mover[i] == 0

# Solve
model.solve(pulp.PULP_CBC_CMD(msg=False))

# Extract results
assignment = []
for i in df.index:
    if pulp.value(x_cabin[i]) == 1:
        assignment.append('Cabin')
    elif pulp.value(x_checkin[i]) == 1:
        assignment.append('Check-in')
    else:
        assignment.append('Movers')
df['assignment'] = assignment

# Compute used capacities
used_cabin_w = sum(df.loc[i, 'weight'] for i in df.index if assignment[i]=='Cabin')
used_cabin_v = sum(df.loc[i, 'volume'] for i in df.index if assignment[i]=='Cabin')
used_check_w = sum(df.loc[i, 'weight'] for i in df.index if assignment[i]=='Check-in')
used_check_v = sum(df.loc[i, 'volume'] for i in df.index if assignment[i]=='Check-in')
net_value = pulp.value(model.objective)

# Display
print("Assignment per item:")
print(df[['item', 'assignment']].to_string(index=False))
print(f"\nUsed Cabin Weight: {used_cabin_w} kg / {cabin_cap['max_weight']} kg")
print(f"Used Cabin Volume: {used_cabin_v} L / {cabin_cap['max_volume']} L")
print(f"Used Check-in Weight: {used_check_w} kg / {checkin_cap['max_weight']} kg")
print(f"Used Check-in Volume: {used_check_v} L / {checkin_cap['max_volume']} L")
print(f"\nMaximum Net Value (₹): {net_value:.2f}")

Assignment per item:
               item assignment
             Laptop      Cabin
         Smartphone      Cabin
           Backpack     Movers
          Desk Lamp     Movers
          Bookshelf     Movers
        Study Table     Movers
              Chair     Movers
           Mattress     Movers
       Bedsheet Set   Check-in
             Pillow   Check-in
     Laundry Basket     Movers
           Wardrobe     Movers
             Mirror   Check-in
        Alarm Clock     Movers
         Headphones     Movers
     Extension Cord     Movers
       Water Bottle     Movers
         Tiffin Box     Movers
   Fan (Table/Wall)   Check-in
     Stationery Set     Movers
Portable Hard Drive     Movers
            Printer   Check-in
    Electric Kettle     Movers
            Shirt 1      Cabin
            Shirt 2      Cabin
            Shirt 3      Cabin
            Shirt 4      Cabin
            Shirt 5      Cabin
             Pant 1      Cabin
             Pant 2      Cabin
             Pant 